In [ ]:
import numpy as np
from scipy import sparse
from math import sqrt
import matplotlib.pyplot as plt
import networkx as nx
import random
import gudhi

In [ ]:
class geometric_network():
    def __init__(self, network_type, size, **kwargs):
        ####################################
        ##### Geometric Network Object #####
        ####################################
        
        ### input -- network_type:
        ##                      2D_lattice: creates 2d lattice of given tiling-can be triangular, square, hexagonal
        #                                   --Additional Arguments--
        #                                     tiling = 3,4,6
        #                                     periodic = bool (connects the edge of the grid to the opposite edge)
        ##                      ring_lattice: creates a ring graph with each node degree 2
        #                                    --Additional Arguments--
        #                                     banded = bool
        #                                     band_length = gonna connect this many closest neigbors (skipping the 
        #                                                   adjacent nodes from both sides because they are already 
        #                                                   connected) from both sides to every node
        #                                     
        #                                     
        ### input -- size: network size, if 2D_lattice, then there will be size^2 many nodes.
        
        ### attribute -- geometric_network.N
        ### attribute -- geometric_network.M
        ### attribute -- geometric_network.graph
        ### attribute -- geometric_network.pos
        ### attribute -- geometric_network.A
        ### attribute -- geometric_network.text 
        
        ### function -- geometric_network.display(n_size, labels = Bool): take a sneakpeak on the networkx object
        ### function -- geometric_network.add_noise_to_geometric(noise_type, d2): add two types of noise
        #                                   either 'ER_like' or 'k_regular' of degree d2
        ### function -- geometric_network.run_excitation(Trials, p, T, seed, threshold): run 
        #                                   the excitation according to these parameters
        
        ### function -- geometric_network.spy_first_activation(first_activation_times): visualize the first
        #                                   activation, input of the functin is the 2nd output of run_excitation
        
        if network_type == '2D_lattice': 
            tiling = kwargs['tiling']
            per = kwargs['periodic']
            if tiling == 3: 
                self.graph = nx.triangular_lattice_graph(size, size, periodic = per, with_positions = True)
                self.pos = nx.get_node_attributes(self.graph,'pos')
                self.M = len(self.graph.edges())
                self.N = len(self.graph.nodes())
                

            elif tiling == 4: 
                self.graph = nx.grid_2d_graph(size, size, periodic = per)
                self.pos = dict( (n, n) for n in self.graph.nodes() )
                self.labels = dict( ((i, j), i * size + j) for i, j in self.graph.nodes() )
                self.M = len(self.graph.edges())
                self.N = len(self.graph.nodes())
                
            elif tiling == 6: 
                self.graph = nx.hexagonal_lattice_graph(size, size, periodic = per, with_positions = True)
                self.pos = nx.get_node_attributes(self.graph,'pos')
                self.M = len(self.graph.edges())
                self.N = len(self.graph.nodes())
            
            
        elif network_type == 'ring_lattice':# TODO: banding for every node
            self.graph = nx.cycle_graph(size)
            theta = (2*np.pi)/size
            self.pos = dict((i,(np.sin(theta*i),np.cos(theta*i))) for i in range(size))
            self.M = len(self.graph.edges())
            self.N = len(self.graph.nodes())
            self.text = 'Ring Lattice'
            if kwargs['banded']:
                if kwargs['band_length'] >= int(self.N/2)-1: raise ValueError('Band length cannot exceed the half of the size of the network')
                if kwargs['band_length'] <2: raise ValueError('Band length should be a positive integer greater 1 since the closest neighbors are already connected')
                for u in range(self.N):
                    for i in range(2,kwargs['band_length']+1):# ranges from 2 to k+2 to avoid the closest node and start
                        ## banding from the second closest node
                        if u + i >= self.N: v = u + i - self.N
                        else: v = u + i
                        self.graph.add_edge(u, v)
                        if u - i < 0: v = self.N + u - i
                        else: v = u - i
                        self.graph.add_edge(u, v)
                self.text = self.text + ' w/ bandlength %d'%kwargs['band_length']
            else:self.text = self.text + ' w/ bandlength 0'
                        
        else: raise ValueError('network type can be a lattice or a ring')
            
        self.A = nx.adjacency_matrix(self.graph)## use .toarray() or .todense() attributes to call the sparse matrix
              
    def display(self, n_size = 15, labels = True):
        
        try: return(nx.draw(self.graph, pos = self.pos, node_size = n_size, with_labels = labels, font_weight = 'bold'))
        
        except: return(nx.draw(self.graph, with_labels = labels, font_weight = 'bold'))
        
    def add_noise_to_geometric(self, noise_type, d2):
        
        # this function adds nongeometric edges to the network that are long range. Depending on the 'noise_type'
        # the way we add these long range edges differ. If noise_type = ER_like, then there will be d2 many
        # non geometric edges ON AVARAGE for every node. When the noise_type = k_regular, every node will 
        # have exactly d2 many long range edges
        
        
        if noise_type == 'ER_like':   
            
            M = int(self.N * d2)
        
            if M%2 == 1: raise ValueError('Try providing an even non-geometric degree')
                
            edges_built = 0
            
            while edges_built <= M/2:
                
                nodes_to_connect = [random.randint(0,self.N-1) for i in range(2)]
                
                if nodes_to_connect[0] == nodes_to_connect[1]: #no self-loops
                    edges_built = edges_built
                elif self.A.toarray()[nodes_to_connect[0]][nodes_to_connect[1]] == 1 or self.A.toarray()[nodes_to_connect[1]][nodes_to_connect[0]] == 1: # no existing loops
                    edges_built = edges_built
                else: 
                    self.graph.add_edge(nodes_to_connect[0],nodes_to_connect[1])
                    edges_built = edges_built + 1
            self.text = self.text + 'and ER-like noise w/ degree %d'%d2

                    
        elif noise_type == 'k_regular':
            
            self.text = self.text + 'and k-regular noise w/ degree %d'%d2
            
            M = int(self.N * d2)
        
            if M%2 == 1: raise ValueError('Try providing an even non-geometric degree')
            
            flag_2 = True
            
            while flag_2:
                flag_2 = False
                #build stubs
                stubs = np.zeros(M)
                for i in range(self.N):
                    index = (i*d2) + np.arange(d2)
                    stubs[index[0]:index[-1]+1] = (i) * np.ones(d2)
                    
                #build undirected link list
                link_list = np.zeros((int(M/2),2))
                for m in range(int(M/2)):
                    flag_1 = True # turn on flag to enter while loop
                    count = 0
                    while flag_1:
                        flag_1 = False #turn off flag to exit while loop
                        rand = [random.randint(0,len(stubs)-1) for i in range(2)]
                        
                        node_A = int(stubs[rand[0]])
                        node_B = int(stubs[rand[1]])
                                                
                        if node_A == node_B: flag_1 = True
                        
                        for n in range(m):
                            if link_list[n,0] == node_A and link_list[n,1] == node_B:
                                flag_1 = True
                            if link_list[n,0] == node_B and link_list[n,1] == node_A:
                                flag_1 = True
                            if self.A.toarray()[node_A][node_B] == 1 or self.A.toarray()[node_B][node_A] == 1:
                                flag_1 = True
                                
                        count = count +1
                        
                        if count > M: flag_2 = True ; break
                            
                    #make link       
                    link_list[m,0] = node_A
                    link_list[m,1] = node_B
                    
                    #remove stubs from list
                    stubs = np.delete(stubs,[rand[0],rand[1]])
            
            #build network
            for m in range(int(M/2)):
                self.graph.add_edge(link_list[m,0],link_list[m,1])
            
        else: raise InputError('Noise type can only be ER_like or k_regular')
            
        self.A = nx.adjacency_matrix(self.graph)
    
    def excitation(self, p, T, seed, threshold, ax = None, spy = False):
        
        n = self.N
        x_1 = sparse.lil_matrix(((n,T+1)), dtype = bool)
        activation_times = np.ones(n, dtype = int)*T
        excited_nodes = self.A[seed].nonzero()[1]
        deg = len(excited_nodes)
        
        ps = np.array([random.uniform(0,1) for i in range(deg)])
        pss = np.array([1-p for i in range(deg)])
        
        vals = ps > pss
        
        for i in range(deg): x_1[excited_nodes[i],0] = vals[i]
            
        t = 0
        all_nodes = set([k for k in range(n)])
        
        while t<T and len(excited_nodes)>0 and np.any(activation_times==T):
            x_old = x_1.toarray()[:,t]
            excited_nodes = np.nonzero(x_old)[0]         
            activation_times[excited_nodes] = np.minimum(activation_times[excited_nodes],np.array([t]*len(excited_nodes)))
            activation_times = activation_times.astype(np.int32)
            
            t+=1
            for j in list(all_nodes-set(excited_nodes)): 
                ids = np.nonzero(self.A[j])[1]
                rands = [random.uniform(0,1) for i in range(len(ids))]
                summ = 0 
                
                for y,e in enumerate(ids):
                    if x_old[e]*rands[y] > 1-p:
                        summ = summ + x_old[e]*rands[y]
                        
                x_1[j,t] = bool(summ/len(ids)>=threshold)
                
        if spy: ax.imshow(x_1.todense())
        
        if np.any(activation_times == T): 
            activation_retry = self.excitation(p, T, seed, threshold, ax = ax, spy = spy)
            return(activation_retry)
            
        else: return(activation_times)
    
    
    def run_excitation(self, Trials, p, T, seed, threshold, ax = None):
        
        n = self.N
        
        first_excitation_times = np.zeros((n,Trials))
        
        if ax is not None: 
            for i in range(Trials):
    
                first_exct = self.excitation(p = p, T = T, seed = seed, threshold = threshold, ax = ax[i], spy = True)
                first_excitation_times[:,i] = first_exct
                ax[i].set_title('Trial %d for %s'%(i+1, self.text), fontsize = 25)
                ax[i].set_xlabel('Time', fontsize = 35)
                ax[i].set_ylabel('Node ID',fontsize = 35)
                ax[i].set_xticks([5*t for t in range(int(T/5)+1)])
                ax[i].set_yticks([10*t for t in range(int(n/10))])
                ax[i].tick_params('both', which = 'major', labelsize = 15)
            plt.tight_layout()
            
        else:
            for i in range(Trials):
    
                first_exct = self.excitation(p = p, T = T, seed = seed, threshold = threshold)
                first_excitation_times[:,i] = first_exct
        
        return(first_excitation_times)
    

    def spy_first_activation(self, first_activation_times):
        
        fig,ax = plt.subplots(1,1,figsize = (10,8))
        
        pos = ax.imshow(first_activation_times, interpolation = 'nearest', aspect = 'auto', cmap = 'viridis',extent = [0.5,Trials+0.5,0,n])
        ax.set_title('First Activation Times for %s'%self.text, fontsize = 20)
        ax.set_xlabel('Trials', fontsize = 15)
        ax.set_ylabel('Node ID', fontsize = 15)
        ax.set_xticks([i for i in range(1,Trials+1)])
        
        fig.colorbar(pos, ax = ax)
        
    def average_over_trials(self, first_activation_times):return(np.mean(first_activation_times, axis = 1))
    
    def make_distance_matrix(self, p, T, threshold, Trials, spy_distance = False):
        n = self.N
        D = np.zeros((n,n))
        
        for i in range(n):
            D[i] = self.average_over_trials(self.run_excitation(Trials, p, T, i, threshold ))
        
        if spy_distance: 
            fig,ax = plt.subplots(1,1, figsize = (15*int(n/20),10*int(n/20)))
            pos = ax.imshow(D, origin = 'lower', interpolation = 'nearest', aspect = 'auto', cmap = 'viridis', extent = [-0.5,n-0.5,-0.5,n-0.5])
            ax.set_title('Distance Matrix for %s'%self.text, fontsize = 100)
            ax.set_ylabel('Excitation starting from node i', fontsize = 95)
            ax.set_xlabel('First time node j gets activated', fontsize = 95)
            ax.tick_params(axis = 'both', labelsize = 80)
            ax.set_xticks([i*5 for i in range(int(n/5))])
            ax.set_yticks([i*5 for i in range(int(n/5))])

        cbar = fig.colorbar(pos, ax = ax)
        cbar.ax.tick_params(labelsize = 80) 
        return(D)
    
    def compute_persistence(self, distances, spy = False):    
        rips_complex = gudhi.RipsComplex(distance_matrix = distances/np.max(distances), max_edge_length = 1)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension = 2)
        diag = simplex_tree.persistence(min_persistence = 0.0)
        if spy: 
            gudhi.plot_persistence_diagram(diag)
            #plt.savefig('/Users/bengieru/persistence.pdf')
        return(diag)
    
    def one_d_Delta(self, persistence):
        oned_holes = [(0,0)]
        for i in range(len(persistence)):
            if persistence[i][0] == 1:
                oned_holes.append(persistence[i][1])
        oned_holes = oned_holes
        oned_holes = np.array(oned_holes)
        persistence_life_times = oned_holes[:,1]-oned_holes[:,0]
        Delta = np.sort(persistence_life_times)[-1]-np.sort(persistence_life_times)[-2]
        return(Delta)

In [ ]:
n = 200
p = 0.5 # cascade spreading probablioty
T = 500 # number of iterations
thresholds = np.linspace(0.05,0.6,12) # resistence of the node to it's neighbors' excitation level
Trials = 10 # number of trials
d2s = [1,2,3,4,5,6,7,8]
bands = 2
all_deltas = {}

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.05/'
networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[0], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[0]] = deltas

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[0] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.1/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[1], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[1]] = deltas

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[1] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.15/'

networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[2], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[2]] = deltas

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[2] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.2/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
    
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[3], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[3]] = deltas

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[3] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.25/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[4], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[4]] = deltas

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[4] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.3/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[5], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[5]] = deltas

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[5] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.35/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[6], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d_thresh=%.2f'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d_thresh=%.2f'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[6]] = deltas  

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[6] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.4/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[7], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[7]] = deltas

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[7] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.45/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[8], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[8]] = deltas  

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[8] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.5/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[9], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[9]] = deltas   

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[9] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.55/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[10], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[10]] = deltas   

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[10] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
networks = []
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/thresh=0.6/'


networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
for i, d2 in enumerate(d2s):
    networks.append(geometric_network('ring_lattice', size = n, banded = True, band_length = bands))
    networks[i+1].add_noise_to_geometric('k_regular', d2)
deltas = []
distance_matrices = []

for i, d2 in enumerate([0]+d2s):
    distance_matrices.append(networks[i].make_distance_matrix(p, T, thresholds[11], Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_d2=%d'%d2 + '.pdf')
    pers = networks[i].compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_d2=%d'%d2 + '.pdf')
    delta = networks[i].one_d_Delta(pers)
    deltas.append(delta)
    
all_deltas['%.2f'%thresholds[11]] = deltas   

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array([0]+d2s)/(2*bands), deltas, label = 'Threshold = %.2f'%thresholds[11] )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize = (10,10))
for i,e in enumerate(thresholds):
    ax.plot(np.array([0]+d2s)/(2*bands), all_deltas['%.2f'%e], label = 'Threshold = %.2f'%e )
fig.suptitle('n = %d, GD = %d, T = %d, Trials = %d, p = %.2f'%(n, 2*bands, T , Trials, p))
ax.set_title('Delta vs nGD/GD for varying Thresholds')
ax.set_xlabel('nGD/GD')
ax.set_ylabel('Delta')
ax.legend(fontsize = 'large')
plt.savefig('/projects/academic/smuldoon/bengieru/excitations/vary_threshold/deltavsngdgd_vary_threshold.pdf')

In [ ]:
bands = 3
d2 = 2
T = 10
thresholds = np.linspace(0.05,0.6,12) # resistence of the node to it's neighbors' excitation level
path = '/projects/academic/smuldoon/bengieru/excitations/vary_threshold/reproduce/'


network = geometric_network('ring_lattice', size = n, banded = True, band_length = bands)
network.add_noise_to_geometric('k_regular', d2)

deltas = []
distance_matrices = []

for i, threshold in enumerate(thresholds):
    distance_matrices.append(network.make_distance_matrix(p, T, threshold, Trials, spy_distance = True))
    plt.savefig(path + 'distance_matrix_threshold=%d'%threshold + '.pdf')
    pers = network.compute_persistence(distance_matrices[i], spy = True)
    plt.savefig(path + 'persistence_diagram_threshold=%d'%threshold + '.pdf')
    delta = network.one_d_Delta(pers)
    deltas.append(delta)
    

fig,ax = plt.subplots(1, 1, figsize = (10,10))    
ax.plot(np.array(thresholds), deltas)
fig.suptitle('GD = %d, nGD=%d, n,T=%d,%d, Trials = %d, p = %.2f'%(bands, d2, n, T , Trials, p))
ax.set_title('Delta vs Thresholds')
ax.set_xlabel('Thresholds')
ax.set_ylabel('Delta')
plt.savefig(path + 'delta_vs_ng_dgd.pdf')